# Real QCNN Training

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [2]:
from qiskit import IBMQ

# KQC Token
TOKEN = "b5b31be2bb5237d52e8bdef143913cd62425eadcbda5c56c6f8ed8d067573ac0e07aa41d23d0bfcc880882a4142e2f812dad876fb3208d5c5f8f321be6626cce"
# DKP Token
#TOKEN = "3ae7ac10f40eb88c7ebb0eca20aa0788e7a96da729e2c3848d9864684362aaf50dfdda7cb3ec1ffd75eb7ed5b44f7c14f2f17419cf600ce14437ee7cd00ac75b"


IBMQ.save_account(TOKEN, overwrite=True)
IBMQ.load_account() # Load account from disk


#KQC Provider
provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='reservations')
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-research', project='kernel-experimen')


#SKKU Provider
#provider = IBMQ.get_provider(hub='ibm-q-skku', group='internal', project='reservations')


backend = "ibmq_toronto"

dev = qml.device(
    'qiskit.ibmq',
    wires=4,
    shots=1024,
    backend=backend,
    provider=provider
)


PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_48.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_48.pt'


# Real QCNN

Hyperparameters

In [3]:
steps = 50
learning_rate = 0.1
batch_size = 10
ansatz = 'TTN'

QCNN

In [4]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [5]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    
@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history, weights_history = [], []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        weights_history.append(weights)
        print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights_history

# Model 2

In [6]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_2430/461388107.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/torch/csrc/utils/tensor_new.cpp:233.)
  X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)


Not pre-trained

In [ ]:
loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)

In [ ]:
f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_not_trained.txt', 'a')
f.write(str(loss_not_trained))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_not_trained.txt', 'a')
f.write(str(weight_not_trained))
f.close()

Model 2

In [7]:
loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_2430/1143620874.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


iteration:  0  cost:  0.50126953125
iteration:  1  cost:  0.503125
iteration:  2  cost:  0.49365234375
iteration:  3  cost:  0.49462890625
iteration:  4  cost:  0.48818359375
iteration:  5  cost:  0.493359375
iteration:  6  cost:  0.48974609375
iteration:  7  cost:  0.4939453125
iteration:  8  cost:  0.47607421875
iteration:  9  cost:  0.47939453125
iteration:  10  cost:  0.47080078125
iteration:  11  cost:  0.47763671875
iteration:  12  cost:  0.45986328125
iteration:  13  cost:  0.46533203125
iteration:  14  cost:  0.45
iteration:  15  cost:  0.44306640625
iteration:  16  cost:  0.444921875
iteration:  17  cost:  0.4498046875
iteration:  18  cost:  0.451171875
iteration:  19  cost:  0.44091796875
iteration:  20  cost:  0.4654296875
iteration:  21  cost:  0.43505859375
iteration:  22  cost:  0.4326171875
iteration:  23  cost:  0.4189453125
iteration:  24  cost:  0.433203125
iteration:  25  cost:  0.41962890625
iteration:  26  cost:  0.40048828125
iteration:  27  cost:  0.39931640625
i

In [8]:
f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_Model2.txt', 'a')
f.write(str(loss_Model2))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_Model2.txt', 'a')
f.write(str(weight_Model2))
f.close()

Model3

In [9]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [10]:
loss_Model3, weight_Model3 = circuit_training(X_train, Y_train, 'Model3')

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_2430/1143620874.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model3(torch.tensor(x))


In [ ]:
f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_Model3.txt', 'a')
f.write(str(loss_Model3))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_Model3.txt', 'a')
f.write(str(weight_Model3))
f.close()

In [ ]:
dev_sim = qml.device('default.qubit', wires=4)
@qml.qnode(dev_sim)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))

PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_48.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_48.pt'

def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


accuracies_Model2 = [], [], []

weights_Model2 = [0.06837866, 0.22757081, 0.66482471, 0.46312987]

prediction_Model2 = [Noisy_QCNN_classifier(weights_Model2, x, Trained='Model2') for x in X_test]
    

accuracy_Model2 = accuracy_test(prediction_Model2, Y_test)



In [ ]:
accuracy_Model2